## Scratch


In [1]:
def f():
    pass

In [6]:
def deco(f):
    def inner(*args, **kwargs):
        print("in")
        result = f()
        print("out")
        return result
    return inner

In [10]:
@deco
def f():
    print("f")

In [8]:
f()

in
f
out


In [11]:
import time

In [70]:
def retry(count=3, sleep=0):
    def deco(f):
        def inner(*args, **kwargs):
            for _ in range(count):
                try:
                    print("trying ...")
                    return f(*args, **kwargs)
                except Exception as exc:
                    time.sleep(sleep)
                    last_exc = exc
            raise last_exc
        return inner
    return deco

In [71]:
import random

In [74]:
@retry(count=2)
def f():
    if random.random() > 0.5:
        raise Exception("failed")
    return 0

In [75]:
f()

trying ...


0

### Generators

In [77]:
class InfiniteIter:
    
    def __iter__(self):
        return self
    
    def __next__(self):
        return 42

In [78]:
it = InfiniteIter()

In [96]:
def gen():
    i = 0
    while i < 3:
        i += 1
        yield i
        

In [97]:
gen()

<generator object gen at 0x7f4f8841c510>

In [81]:
type(gen)

function

In [82]:
g = gen()

In [83]:
type(g)

generator

In [84]:
for v in g:
    print(v)

1
2
3


In [85]:
import os

In [86]:
os.walk?

In [88]:
for v in gen():
    print(v)

1
2
3


In [91]:
# for i in range(10000): # Python 2 would allocate an array for that, "xrange"
#     pass

In [92]:
range?

Generator Pipelines

* generator expression (~ list comprehension)

In [93]:
numbers = (i for i in range(1, 10))
squared = (i * i for i in numbers)
filtered = (i for i in squared if i % 7 == 0)

In [94]:
for v in filtered:
    print(v)

49


* iterating over a list of files, e.g. to resize images with a boundary on memory

Advantages:

* memory efficient
* extendable


In [95]:
zip(range(10), range(10))

### Context Manager

* resource setup and teardown helper
* pep 343
* enter, exit

In [98]:
with open("Scratch.ipynb") as f:
    print(len(f.read()))

8326


Use cases:
    
* files
* temp files
* database connections
* timing
* temporary state changes
* ...

In [107]:
class A:
    def __enter__(self):
        print("enter")
        return 123
    
    def __exit__(self, exc, exc_type, traceback):
        print("exit")

In [108]:
with A() as obj:
    print(f"inside with {obj}")

enter
inside with 123
exit


In [121]:
class A:
    def __enter__(self):
        pass
    
    def __exit__(self, exc, exc_type, traceback):
        print(exc, exc_type, traceback)
        # return True # True will suppress the exception

In [124]:
with A():
    raise Exception("failed")

<class 'Exception'> failed <traceback object at 0x7f4f9833d480>


Exception: failed

In [125]:
import contextlib

In [127]:
@contextlib.contextmanager
def f(*args, **kwargs):
    print("setup")
    try:
        yield 123
    finally:
        print("cleaning up")

In [129]:
with f() as v:
    print("inside with ...")
    print(v)

setup
inside with ...
123
cleaning up


In [130]:
with contextlib.suppress(FileNotFoundError):
    os.remove("someotherfile.tmp")

In [132]:
class mycontext(contextlib.ContextDecorator):
    def __enter__(self):
        print("enter")
        return self
    
    def __exit__(self, exc, exc_type, tb):
        print("exit")
        return False

In [135]:
@mycontext()
def f():
    print("middle")

In [136]:
f()

enter
middle
exit


### Functional aspect

* itertools (sequences)
* map, reduce (functools.reduce)
* first class functions (e.g. sorted key=...)
* operator module
* functools.partial

In [137]:
import operator

In [138]:
operator.add(1, 2)

3

In [139]:
a = list(range(10))
b = list("abcdef")

In [140]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [141]:
b

['a', 'b', 'c', 'd', 'e', 'f']

In [142]:
zip(a, b)

In [143]:
list(zip(a, b))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [144]:
import itertools

In [145]:
list(itertools.zip_longest(a, b))

[(0, 'a'),
 (1, 'b'),
 (2, 'c'),
 (3, 'd'),
 (4, 'e'),
 (5, 'f'),
 (6, None),
 (7, None),
 (8, None),
 (9, None)]

Map and reduce

In [149]:
list(map(len, ["abc", "de", "wer"]))

[3, 2, 3]

In [150]:
[len(v) for v in ["abc", "de"]]

[3, 2]

In [151]:
import functools

In [153]:
functools.reduce(operator.add, [1, 2, 3], 0)

6

Composition

In [157]:
list(map(sum, zip(range(10), range(10))))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Grouping elements

In [161]:
def grouper(inputs, n, fillvalue=None):
    iters = [iter(inputs)] * n
    return itertools.zip_longest(*iters, fillvalue=fillvalue)

In [164]:
list(grouper(range(10), 2))

[(0, 1), (2, 3), (4, 5), (6, 7), (8, 9)]

Product

In [168]:
for i in itertools.product(["A", "B"], range(4), ["x", "y"]):
    print(i)

('A', 0, 'x')
('A', 0, 'y')
('A', 1, 'x')
('A', 1, 'y')
('A', 2, 'x')
('A', 2, 'y')
('A', 3, 'x')
('A', 3, 'y')
('B', 0, 'x')
('B', 0, 'y')
('B', 1, 'x')
('B', 1, 'y')
('B', 2, 'x')
('B', 2, 'y')
('B', 3, 'x')
('B', 3, 'y')


In [170]:
list(itertools.takewhile(lambda x: x < 8, range(20)))

[0, 1, 2, 3, 4, 5, 6, 7]

In [171]:
# dropwhile ...

Selector filter

In [172]:
s = "abc"

In [173]:
sel = [0, 1, 1]

In [175]:
list(itertools.compress(s, sel))

['b', 'c']

Groupby function

In [180]:
numbers = sorted([str(i) for i in range(30)])

In [181]:
numbers

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [ ]:
for k, g in itertools.groupby(numbers, key=lambda v: v[0]):
    print(k, g)
    for v in g:
        print(f"    {v}")

Task: Group names by vowel combinations (e.g. all names having two "a" vowels, like "anna", "hanna", ... belong to the same group) - print out the size of each group.)

https://raw.githubusercontent.com/miku/miscpy/main/Extra/names.tsv

In [186]:
import urllib

In [189]:
names = urllib.request.urlopen("https://raw.githubusercontent.com/miku/miscpy/main/Extra/names.tsv").readlines()

In [191]:
names[:10]

[b'aaden\n',
 b'aaliyah\n',
 b'aarav\n',
 b'aaron\n',
 b'ab\n',
 b'abagail\n',
 b'abb\n',
 b'abbey\n',
 b'abbie\n',
 b'abbigail\n']

In [192]:
names = [name.decode("utf-8").strip() for name in names if name.strip()]

In [194]:
len(names)

6783

In [ ]:
names[:10]

* key function (name -> vowels)
* sorted
* itertools.groupby

In [230]:
def vowels(s):
    return "".join([c for c in s if c in list("aeiou")])

In [229]:
vowels("abbalieee")

'aaieee'

In [239]:
vc = {}
for k, g in itertools.groupby(sorted(names, key=vowels), key=vowels):
    # print(k, len(list(g)))
    vc[k] = set(g)

In [242]:
vc["uia"]

{'drucilla',
 'drusilla',
 'julia',
 'julian',
 'juliann',
 'julisa',
 'julissa',
 'junia',
 'justina',
 'lucia',
 'lucian',
 'lucina',
 'lucinda',
 'luisa',
 'sullivan',
 'uriah',
 'urijah',
 'yulisa',
 'yulissa'}

### Collections

* namedtuple, dataclass
* various dict subclasses

Sidenote: shelve

Record like structures

* tuple
* namedtuple
* maybe a dataclass

In [244]:
import collections

In [245]:
Record = collections.namedtuple("Record", "a b c")

In [246]:
type(Record)

type

In [247]:
record = Record(1, 2, 3)

In [249]:
record[0]

1

In [250]:
record.a

1

In [251]:
record._asdict()

{'a': 1, 'b': 2, 'c': 3}

In [259]:
Record = collections.namedtuple("Record", "a b c", defaults=[100])

In [260]:
Record()

TypeError: __new__() missing 2 required positional arguments: 'a' and 'b'

In [261]:
Record(1, 2)

Record(a=1, b=2, c=100)

In [263]:
Record(1, b=2)

Record(a=1, b=2, c=100)

In [264]:
Record(1, 2, 3)

Record(a=1, b=2, c=3)

DataClasses

In [265]:
from dataclasses import dataclass

In [266]:
@dataclass
class InventoryItem:
    name: str
    unit_price: float
    quantity_on_hand: int = 0
        
    def total_cost(self): 
        return self.unit_price * self.quantity_on_hand

In [267]:
InventoryItem.__init__

<function __main__.__create_fn__.<locals>.__init__(self, name: str, unit_price: float, quantity_on_hand: int = 0) -> None>

* init, repr, eq, order, frozen, ...

In [268]:
item = InventoryItem(name="PART123", unit_price=1.0, quantity_on_hand=10)

In [269]:
item

InventoryItem(name='PART123', unit_price=1.0, quantity_on_hand=10)

In [271]:
InventoryItem(name=123, unit_price=1.0, quantity_on_hand=10)

InventoryItem(name=123, unit_price=1.0, quantity_on_hand=10)

In [287]:
from dataclasses import field

In [278]:
from typing import List

In [290]:
@dataclass
class C:
    mylist: List[int] = field(default_factory=list)

In [291]:
c = C()

In [292]:
c.mylist  # TODO: fix this



[]

In [293]:
c.mylist += [1, 2, 3]

In [294]:
c.mylist

[1, 2, 3]

In [295]:
import dataclasses

In [296]:
dataclasses.astuple(c)

([1, 2, 3],)

In [297]:
dataclasses.asdict(c)

{'mylist': [1, 2, 3]}

In [298]:
import json

In [300]:
json.dumps(dataclasses.asdict(c))

'{"mylist": [1, 2, 3]}'

Some custom dictionary subclasses

* OrderedDict

In [301]:
dd = collections.defaultdict(list)

In [303]:
dd["a"].append(123)

In [304]:
dd = collections.defaultdict(123)

TypeError: first argument must be callable or None

In [307]:
dd = collections.defaultdict(lambda: 123)

In [308]:
dd["a"]

123

In [309]:
counter = collections.Counter()

In [310]:
counter["a"] += 1

In [311]:
counter["b"] += 2

In [312]:
counter

Counter({'a': 1, 'b': 2})

In [313]:
counter.most_common()

[('b', 2), ('a', 1)]

In [314]:
from collections import abc

In [315]:
class C(abc.Sequence):
    pass

In [316]:
c = C()

TypeError: Can't instantiate abstract class C with abstract methods __getitem__, __len__

In [320]:
import string

In [321]:
class C(abc.Sequence):
    def __init__(self):
        self.alpha = string.ascii_lowercase
    
    def __getitem__(self, index):
        if index < len(self.alpha):
            return (index, self.alpha[index])
        raise IndexError()
        
    def __len__(self):
        return len(self.alpha)

In [322]:
c = C()

In [323]:
c[4]

(4, 'e')

In [324]:
c[100]

IndexError: 

In [325]:
isinstance(c, abc.Sequence)

True

In [327]:
issubclass(C, abc.Sequence)

True

In [329]:


class ListBasedSet(collections.abc.Set):
    ''' Alternate set implementation favoring space over speed
        and not requiring the set elements to be hashable. '''
    def __init__(self, iterable):
        self.elements = lst = []
        for value in iterable:
            if value not in lst:
                lst.append(value)

    def __iter__(self):
        return iter(self.elements)

    def __contains__(self, value):
        return value in self.elements

    def __len__(self):
        return len(self.elements)

s1 = ListBasedSet('abcdef')
s2 = ListBasedSet('defghi')
overlap = s1 & s2       



In [333]:
print(overlap.elements)

['d', 'e', 'f']


In [334]:
import abc

In [335]:
class DataSource(abc.ABC):
    
    @abc.abstractmethod
    def name(self):
        pass

In [338]:
class Dummy(DataSource):
    def name(self):
        return "dummy"

In [339]:
dummy = Dummy()

In [341]:
isinstance(dummy, DataSource)

True

> Exceptions

* reuse exception is fine

In [345]:
try:
    1 / 0
except ZeroDivisionError as exc:
    print("failed")
    raise

failed


ZeroDivisionError: division by zero

In [346]:
try:
    1 / 0
except Exception:
    print("failed")
    raise

failed


ZeroDivisionError: division by zero

In [351]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

for cls in [B, C, D]:
    try:
        raise cls()
    except D:
        print("D")
    except C:
        print("C")
    except B:
        print("B")
    

B
C
D


* https://third-bit.com/talks/greatest-hits/#1

In [352]:
try:
    raise Exception("spam", "eggs")
except Exception as exc:
    print(type(exc))
    print(exc.args)
    print(exc)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')


In [356]:
def compute(a, b):
    try:
        a / b
    except Exception as exc:
        log(exc)
        err = exc

def log(exc):
    raise IOError("unavailable")

In [357]:
compute(1, 2)

In [358]:
compute(1, 0)

OSError: unavailable

In [359]:
err

NameError: name 'err' is not defined

In [362]:
try:
    compute(1, 0)
except Exception as exc:
    err = exc

In [363]:
err

OSError('unavailable')

In [364]:
err.__cause__

In [365]:
err.__context__

ZeroDivisionError('division by zero')

In [366]:
err.__traceback__

In [367]:
err.__traceback__.tb_frame

<frame at 0x55f4037401d0, file '/tmp/ipykernel_160113/2204511548.py', line 4, code <module>>

In [368]:
err.__traceback__.tb_frame.f_code

<code object <module> at 0x7f4f71e94df0, file "/tmp/ipykernel_160113/2204511548.py", line 1>

In [369]:
err.__traceback__.tb_frame.f_locals

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def f():\n    pass',
  'def deco(f):\n    def inner(*args, **kwargs):\n        print("in")\n        return f()\n        print("out")\n    return inner',
  '@deco\nf()',
  '@deco\ndef f():\n    print("f")',
  'f()',
  'def deco(f):\n    def inner(*args, **kwargs):\n        print("in")\n        result = f()\n        print("out")\n        return result\n    return inner',
  '@deco\ndef f():\n    print("f")',
  'f()',
  '@deco(a=1)\ndef f():\n    print("f")',
  '@deco\ndef f():\n    print("f")',
  'import time',
  'def retry(count=3, sleep=0):\n    def deco(f):\n        def inner(*args, **kwargs):\n            for _ in range(count):\n                try:\n                    return f(*args, kwargs)\n               